<div style="display: inline; justify-content: space-between;">
    <img src="assets/jupyter_logo.png" width="60px;"/>
    <span>&nbsp;</span>
    <img src="assets/cruk_logo.jpg" width="260px" style="padding: 4px"/>
    <span>&nbsp;</span>
    <img src="assets/ioa_logo.png" width="80px"/>
</div>

# STPT Viewer

## Setup paths

In [ ]:
stpt_run = "CM1-190610-1302"
stpt_analysis_path = "/data/meds1_b/processed/STPT/test_eglez/"

## Imports

In [ ]:
import panel as pn
from imaxt_image.viewer.stpt import StptDataViewer, setup_notebook

The following is needed to load the javascript libraries and make them available in the notebook.

In [ ]:
setup_notebook()

## Read data

In [ ]:
dv = StptDataViewer(stpt_run, path=stpt_analysis_path)

### Data size

The imaging data are stored compressed and in multiscale tiles in levels 2^0, 2^1, 2^2, 2^3, 2^4, 2^5

* l.1 : 130 GB  -- this is compressed - real size is about 1TB
* l.2 :  31 GB
* l.4 :   7.6 GB
* l.8 :   2.0 GB
* l.16 :  529 M
* l.32 : 150 M

Total: 170 GB

In [ ]:
dv.ds[1]

In [ ]:
dv.histogram()

In [ ]:
panel = dv.view(
    channels=[2, 3, 4],
    rscale=(0.1, 2),
    gscale=(0.1, 10),
    bscale=(0.1, 10),
    show_miniview=True,
    height=900,
)

In [ ]:
panel.servable()

## Interaction

In [ ]:
dv.goto(10)

### Straight sync version

In [ ]:
for i in range(20):
    dv.goto(i+1)

### AsyncIO -- allows interaction with the figure

In [ ]:
import asyncio

loop = asyncio.get_running_loop()

async def main():
    for i in range(50):
        dv.goto(i+1)
        await asyncio.sleep(1)

coro = asyncio.ensure_future(main())

### Create a movie

In [ ]:
for i in range(100):
    dv.goto(i+1)
    element = dv.main.data[()]
    p = hv.render(element, backend="matplotlib")
    p.axes[0].set_axis_off()
    p.savefig(f"plot_{i:03d}.png", bbox_inches="tight", pad_inches=0, dpi=150)

In [ ]:
!ffmpeg -y -r 1 -i plot_%03d.png -vcodec mpeg4 movie.mp4

### Display defined regions

In [ ]:
import pandas as pd

pd.DataFrame(data=dv.box_stream.data)